In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
import os
import json
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

# Sessão de variaveis
path_to_downloads = "C:\\Users\\MrRobot\\Downloads\\"
path_to_musics = "C:\\Users\\MrRobot\\Music\\"
service = Service(executable_path="./chromedriver.exe")
options = webdriver.ChromeOptions()
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36' # se tiver "headless" no user_agent o site bloqueia o acesso
options.add_argument('user-agent={0}'.format(user_agent))
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--mute-audio')
# options.add_argument("user-data-dir=C:\\Users\\MrRobot\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 4") #Path to your chrome profile
c = webdriver.Chrome(service=service, options=options)
wdw = WebDriverWait(c, 40, 2)

def pegando_urls_no_ytb(link):
    """crawl da playlist do youtube e retorna ela em DICT"""
    try:
        c.get(link)

        sleep(20)
        videos = c.find_elements(By.XPATH, "//a[@id = 'video-title']")
    
        urls = []
        for hrefs in videos:
            dict = {}
            dict['title'] = hrefs.get_attribute("innerHTML").replace('\n          ', '').replace('\n        ', '')
            dict['url'] = hrefs.get_attribute("href")
            urls.append(dict)

        print(f'total de URLs: {len(urls)}')
        return urls
    except:
        return "ERRO ao pegar URLS da playlist no Youtube."

def automacao_only_mp3(url_msc):
        """Recebe uma url de um video e Faz as ações dentro do site de download"""
        site = "https://pt.onlymp3.to/38/"
        c.get(site)

        search_bar = c.find_element(By.ID, "txtUrl")
        search_bar.send_keys(url_msc)
        
        sleep(1)
        convert_btn = c.find_element(By.CLASS_NAME, "start-btn")
        convert_btn.click()

        wdw.until(lambda c: c.find_element(By.XPATH, "//button[@class='btn']"), "Esperando btn de download")

        sleep(1)
        download_btn = c.find_element(By.XPATH, "//button[@class='btn']")
        download_btn.click()
        # print('espera acabou')

def eh_hora_de_reinciar(numero):
    """
        Função que reinicia o browser a cada 10 download

        obs: atrelar a mesma variável utilizada no inicio, nesse caso c = eh_hora_de_reinciar(10)
    """
    mbdir = os.listdir(path_to_downloads)
    mbdir.remove('desktop.ini')
    print('{} músicas sendo baixadas...'.format(numero))

    if numero == 1:
        # espera
        c.quit()
        print('browser reiniciando')
    else:
        print('não é hr de reiniciar o browser')

def espera_download():
    """ Pergunta: Tem algum item em processo de download ou o diretório está vázio ? """
    sleep(3)
    mb = os.listdir(path_to_downloads)
    mb.remove('desktop.ini')
    
    if len(mb) == 0: #se diretório está vazio
        return True
    
    for item in mb:
        if ".crdownload" in item:
            # Se tiver item em download 
            # print(item)
            return True
    return False

def downloads_suspeitos(playlist):
    """ Retorna itens q não foram confirmados na area de download"""
    confirmados = []
    lista_organizada = []
    mb = os.listdir(path_to_downloads)
    mb.remove('desktop.ini')

    for item in playlist:
        for msc in mb:
            if item['title'] in msc:
                confirmados.append(item) # items confirmados

    print('confimados: {}'.format(len(confirmados)))
    for item in confirmados:
        playlist.remove(item)

    for index, item in enumerate(playlist):
        lista_organizada.append(item['title'])
    return playlist, lista_organizada

def renomear_arquivos():
    """Renomeia os arquivos no diretório de downloads"""
    mb = os.listdir(path_to_downloads)
    mb.remove('desktop.ini')

    for i in mb:
        new_i = i.replace('onlymp3.to - ', '')
        os.replace(path_to_downloads+i, f'{path_to_downloads}{new_i[:-35]}.mp3')

def mover_arquivos():
    """mover os arquivos do diretório de downloads para music"""

    mb = os.listdir(path_to_downloads)
    mb.remove('desktop.ini')

    for i in mb:
        os.replace(path_to_downloads+i, f'{path_to_musics}{i}')

def download_404():
    """
        Pergunta: ao clicar em download foi redirecionado para uma página de 404 ?
    """
    try:
        pagina_404 = c.find_element(By.XPATH, '//body[@id = "t"]')
        if bool(pagina_404):
            return True
    except: 
        return False


In [2]:
# PROCESSO DE EXECUÇÃO:
link_playlist_ytb = 'https://www.youtube.com/playlist?list=PLB5S3eu0mZof9705EtpG8SeFACN2itGRP'

if 'urls_ytb' in globals(): # se a variavel já foi declarada...
    if len(urls_ytb) > 0:
        print(f'Não precisará fazer scrapy no ytb... Qt de Urls: {len(urls_ytb)}')
    else: # senão tem itens na lista...
        urls_ytb = pegando_urls_no_ytb(link_playlist_ytb)
else: # senão foi declarada a variavel
    urls_ytb = pegando_urls_no_ytb(link_playlist_ytb)

contador = 0
tentativas_download = 1
bad_request_404 = []
while bool(urls_ytb) and tentativas_download < 5: # vai tentar 4x enquanto ter item na lista
    print(f'iniciando a {tentativas_download}° Tentativa')
    for msc in urls_ytb:
        try:
            automacao_only_mp3(msc['url'])

            # Enquanto espera download... 
            timeout = 0
            while espera_download():
                timeout += 1
                sucesso_no_download = True
                if download_404():
                    # Quando der 404 ele pula pra próx musica
                    print(f'Problema de servidor na música {msc["title"]}')
                    bad_request_404.append(msc)
                    sucesso_no_download = False
                    break
                if timeout > 6:
                    # Vai dar timeout se nao fizer o download em até 60 segundos
                    sucesso_no_download = False
                    break
                sleep(10)

            if sucesso_no_download:
                urls_ytb.remove(msc)
                renomear_arquivos()
                mover_arquivos()
                print(f"Download concluído: {msc['title']}")
                contador += 1

            # Reiniciar browser quando quantidade de downloads for um divisor de 10
            dez_msc = contador % 10
            if dez_msc == 0 and contador != 0:
                c.quit()
                c = webdriver.Chrome(service=service, options=options)
                wdw = WebDriverWait(c, 20, 2)

            print(f'contador: {contador}. Número de itens para ser baixados: {len(urls_ytb)}')
        except:
            print(f'Erro na música: {msc["title"]}')
    tentativas_download += 1

c.quit()

total de URLs: 5
iniciando a 1° Tentativa
Download concluído: Guilherme e Benuto feat @Hugo e Guilherme  - Haja Colírio | DVD Deu Rolo
contador: 1. Número de itens para ser baixados: 4
Download concluído: Gusttavo Lima - Bloqueado (Buteco In Boston)
contador: 2. Número de itens para ser baixados: 3
Download concluído: Guilherme e Benuto - 3 Batidas (DVD AMANDO BEBENDO E SOFRENDO)
contador: 3. Número de itens para ser baixados: 2
iniciando a 2° Tentativa
Download concluído: Luan Santana - ERRO PLANEJADO feat Henrique e Juliano (LUAN CITY)
contador: 4. Número de itens para ser baixados: 1
iniciando a 3° Tentativa
Download concluído: Jorge &amp; Mateus - Então Valeu (Clipe Oficial) [É Simples Assim]
contador: 5. Número de itens para ser baixados: 0


In [3]:
print('urls q não foram baixadas:')
urls_ytb

urls q não foram baixadas:


[]

In [4]:
print('Falha na requisição: ')
bad_request_404

Falha na requisição: 


[]